In [9]:
%load_ext autoreload

%autoreload 2
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow.keras.callbacks as cb
import time
import numpy as np
from data import get_datasets
import matplotlib.pyplot as plt
import os
import shutil
from functools import partial
from data import get_datasets
from experiment import create_model, evaluate, find_adv_k, sentiment
import model_adv

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
(train_dataset, test_dataset), info = get_datasets(batch_size=1)
encoder = info.features['text'].encoder
vocab_size=info.features['text'].encoder.vocab_size 

In [11]:
transformer, _optimizer, _checkpoint, _manager = create_model(models_dir="models/pos_enc_True", load_checkpoint=True, 
                           vocab_size=vocab_size, use_positional_encoding=True, run_eagerly=True)

# adv = model_adv.create_model(models_dir="adv", load_checkpoint=False, run_eagerly=True, d_model=512)

Loaded previously trained model.


In [5]:
# Initial accuracy
metrics = evaluate(transformer, test_dataset)
print(metrics.result().numpy())

KeyboardInterrupt: 

In [12]:
EPOCHS = 50
D_MODEL = 128

adv_model, adv_optimizer = model_adv.create_model(models_dir="adv", load_checkpoint=False, 
                                            run_eagerly=True, d_model=D_MODEL)


In [13]:
x, y = next(iter(train_dataset))
#     for batch, (x, y) in enumerate(train_dataset):
loss_t, k, best_loss_t, best_adv_k = find_adv_k(x, y, transformer, adv_model, adv_optimizer, epochs=50)

Returned k: [  1   4 108 128]
[0] Loss_t = 0.008126692846      
[0] Loss_t (custom k) = 0.008126692846      
[1 ] Loss_t = 0.006112156436        Loss_k = -11.30030251           Loss = -5.647095203        
[2 ] Loss_t = 0.005334352609        Loss_k = -11.45360088           Loss = -5.724133492        
[3 ] Loss_t = 0.004694001749        Loss_k = -11.60806656           Loss = -5.801686287        
[4 ] Loss_t = 0.004172470886        Loss_k = -11.7632246            Loss = -5.879526138        
[5 ] Loss_t = 0.003728145733        Loss_k = -11.91905499           Loss = -5.957663536        
[6 ] Loss_t = 0.003343239427        Loss_k = -12.07547283           Loss = -6.036064625        
[7 ] Loss_t = 0.003003796563        Loss_k = -12.23269844           Loss = -6.114847183        
[8 ] Loss_t = 0.002703865292        Loss_k = -12.39034653           Loss = -6.19382143         
[9 ] Loss_t = 0.002436336828        Loss_k = -12.54795265           Loss = -6.272758007        
[10] Loss_t = 0.00219760113

In [14]:
%%javascript
require.config({
  paths: {
      d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min',
      jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
  }
});

<IPython.core.display.Javascript object>

In [19]:
sentence = "This is a very terrible movie . I will never watch this bad movie again ."
# sentence = "this was a perfect movie . I enjoyed it a lot"
tokens = sentence.split()

sent, weights = sentiment(sentence, encoder=encoder, adv_model=adv_model, transformer=transformer)
# sent, weights = sentiment(sentence, encoder=encoder, transformer=old_transformer)
sent

'neg'

In [10]:
metrics = evaluate(transformer, test_dataset, adv_model=adv)
print(metrics.result().numpy())

0.79986244


In [34]:
print(tf.norm(k).numpy())
print(tf.norm(best_adv_k).numpy())

214.1467
79561.1


In [14]:
print(f'Original loss_t: {loss_t}')
print(f'Best loss_t:     {best_loss_t}')

Original loss_t: 0.0007400644826702774
Best loss_t:     0.00014332833234220743


In [24]:
print(tf.reshape(k[0][0][0],(-1, 4)))
print(tf.reshape(best_adv_k[0][0][0], (-1, 4)))

tf.Tensor(
[[-1.0622562  -0.23984744  1.0748044   1.3748744 ]
 [-0.69511235  0.541495   -0.26275066  0.00983193]
 [ 0.587861    0.8562857   0.37515846  0.7165856 ]
 [-0.8721777  -0.08632609 -0.13359296 -0.27416104]
 [-0.9817763  -0.07322308 -0.07859396  1.1635443 ]
 [-0.5991031  -0.3586921  -0.63432235 -1.2879654 ]
 [-1.3708589   0.6427558   0.01798093 -0.36737213]
 [ 2.1075368   0.32159662  0.22088861  0.41846114]
 [ 0.18461812 -0.52916014  1.0895694  -1.4266332 ]
 [-0.21394011 -0.13209644 -0.45749953  0.2583496 ]
 [ 0.8876147   0.55790997  0.8856335   0.17673488]
 [ 1.8994558   0.9954314   1.8560383   1.3102771 ]
 [-0.1030802   0.2939429   1.0747086   0.5858982 ]
 [ 1.9763365  -0.88368344  2.5498774  -0.2831556 ]
 [ 1.8231454   0.20237808 -0.8465917   0.2549476 ]
 [-1.0648656  -1.008372    1.7104899   0.34775013]
 [-0.35755393 -1.0473596  -0.05636305  1.8079398 ]
 [-0.05273503 -0.01887925 -1.3248074  -1.0591247 ]
 [-0.7161503   0.5835323   2.18067     0.10653253]
 [ 1.4768964  -0.481

In [11]:
np.save('adv_results/x', x.numpy())
np.save('adv_results/y', y.numpy())
np.save('adv_results/best_adv_k', best_adv_k.numpy())
np.save('adv_results/k', k.numpy())
np.save('adv_results/loss_t', loss_t.numpy())
np.save('adv_results/best_loss_t', best_loss_t.numpy())

In [12]:
# Load the best values
np.load('adv_results/x.npy')
np.load('adv_results/y.npy')
np.load('adv_results/best_adv_k.npy')
np.load('adv_results/k.npy')
print(np.load('adv_results/loss_t.npy'))
print(np.load('adv_results/best_loss_t.npy'))

0.024033187
0.01337338
